In [1]:
import torch
import pandas as pd, numpy as np
from glob import glob
from tqdm.notebook import tqdm
import os
import seaborn as sns
from pylab import rcParams
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import register_matplotlib_converters
from torch import nn, optim
import torch.nn.functional as F

In [2]:
dfA = []
for path in tqdm(glob('./data/raw/training_setA/training/*.psv')):
    df = pd.read_csv(path, sep="|")
    pid = int(path.split("\\")[1][1:-4])
    df['pid']=pid
    dfA.append(df)

dfB=[]
for path in tqdm(glob('./data/raw/training_setB/training_setB/*.psv')):
    df = pd.read_csv(path, sep="|")
    pid = int(path.split("\\")[1][1:-4])
    df['pid'] = pid
    dfB.append(df)  

  0%|          | 0/20336 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

In [3]:
dfA = pd.concat(dfA)
dfB = pd.concat(dfB)

In [4]:
pd.set_option('display.max_columns',None)

In [5]:
dfA

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,pid
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,1,0,1
1,97.0,95.0,NaN,98.0,75.33,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,2,0,1
2,89.0,99.0,NaN,122.0,86.00,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,3,0,1
3,90.0,95.0,NaN,NaN,NaN,NaN,30.0,NaN,24.0,NaN,NaN,7.36,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,4,0,1
4,103.0,88.5,NaN,122.0,91.33,NaN,24.5,NaN,NaN,NaN,0.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,88.0,98.0,NaN,135.0,81.00,64.0,16.0,NaN,NaN,NaN,0.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.29,1,NaN,NaN,-0.03,31,1,20643
29,96.0,98.0,38.72,174.0,97.00,72.0,16.0,NaN,2.0,NaN,NaN,7.48,34.0,97.0,90.0,26.0,107.0,NaN,NaN,2.5,NaN,NaN,NaN,2.2,3.5,3.9,0.9,NaN,27.8,NaN,NaN,NaN,NaN,NaN,62.29,1,NaN,NaN,-0.03,32,1,20643
30,140.0,97.0,NaN,133.0,81.50,62.5,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.29,1,NaN,NaN,-0.03,33,1,20643
31,120.0,96.0,NaN,154.0,118.00,105.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.29,1,NaN,NaN,-0.03,34,1,20643


In [6]:
dfB

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,pid
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,1,1.0,0.0,-214.64,1,0,100001
1,93.0,92.5,NaN,110.0,76.0,56.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,1,1.0,0.0,-214.64,2,0,100001
2,91.0,96.0,NaN,108.0,84.5,72.0,23.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,233.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,1,1.0,0.0,-214.64,3,0,100001
3,93.0,98.0,NaN,123.0,87.0,61.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,1,1.0,0.0,-214.64,4,0,100001
4,93.0,95.0,NaN,110.0,81.0,70.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,1,1.0,0.0,-214.64,5,0,100001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,80.0,96.0,NaN,115.0,87.0,65.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62,0,NaN,NaN,0.00,31,0,120000
31,74.0,97.0,NaN,114.0,83.0,67.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62,0,NaN,NaN,0.00,32,0,120000
32,78.0,98.0,NaN,110.0,83.0,69.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62,0,NaN,NaN,0.00,33,0,120000
33,82.0,99.0,36.6,124.0,91.0,71.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62,0,NaN,NaN,0.00,34,0,120000


In [7]:
df = pd.concat([dfA,dfB]).reset_index(drop=True)

In [8]:
df

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,pid
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,1,0,1
1,97.0,95.0,NaN,98.0,75.33,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,2,0,1
2,89.0,99.0,NaN,122.0,86.00,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,3,0,1
3,90.0,95.0,NaN,NaN,NaN,NaN,30.0,NaN,24.0,NaN,NaN,7.36,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,4,0,1
4,103.0,88.5,NaN,122.0,91.33,NaN,24.5,NaN,NaN,NaN,0.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552205,80.0,96.0,NaN,115.0,87.00,65.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.00,0,NaN,NaN,0.00,31,0,120000
1552206,74.0,97.0,NaN,114.0,83.00,67.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.00,0,NaN,NaN,0.00,32,0,120000
1552207,78.0,98.0,NaN,110.0,83.00,69.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.00,0,NaN,NaN,0.00,33,0,120000
1552208,82.0,99.0,36.6,124.0,91.00,71.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.00,0,NaN,NaN,0.00,34,0,120000


### train, test 분리

In [9]:
# 3만명의 샘플을 가져온다.
train_pids = df.pid.sample(30000, replace=False, random_state=7777)

train_pids = np.random.choice(df.pid.unique(), 30000, replace=False)
train_df = df.query('pid in @train_pids').reset_index(drop=True)
test_df = df.query('pid not in @train_pids').reset_index(drop=True)

In [10]:
train_df

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,pid
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,1,0,1
1,97.0,95.0,NaN,98.0,75.33,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,2,0,1
2,89.0,99.0,NaN,122.0,86.00,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,3,0,1
3,90.0,95.0,NaN,NaN,NaN,NaN,30.0,NaN,24.0,NaN,NaN,7.36,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,4,0,1
4,103.0,88.5,NaN,122.0,91.33,NaN,24.5,NaN,NaN,NaN,0.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156567,80.0,96.0,NaN,115.0,87.00,65.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.00,0,NaN,NaN,0.00,31,0,120000
1156568,74.0,97.0,NaN,114.0,83.00,67.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.00,0,NaN,NaN,0.00,32,0,120000
1156569,78.0,98.0,NaN,110.0,83.00,69.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.00,0,NaN,NaN,0.00,33,0,120000
1156570,82.0,99.0,36.6,124.0,91.00,71.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.00,0,NaN,NaN,0.00,34,0,120000


In [11]:
test_df.iloc[60:100,:]

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,pid
60,95.0,90.5,NaN,121.0,80.0,63.0,17.00,NaN,-3.0,NaN,1.00,7.28,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.92,1,NaN,NaN,-0.03,13,0,9
61,96.5,89.5,36.44,109.5,80.5,67.5,16.00,NaN,-1.0,NaN,NaN,7.30,52.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.92,1,NaN,NaN,-0.03,14,0,9
62,104.0,91.0,NaN,96.0,66.0,54.0,16.00,NaN,-3.0,NaN,NaN,7.29,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.4,NaN,NaN,NaN,NaN,NaN,39.5,NaN,NaN,NaN,NaN,NaN,27.92,1,NaN,NaN,-0.03,15,0,9
63,107.0,94.0,NaN,106.0,72.5,61.0,35.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.92,1,NaN,NaN,-0.03,16,0,9
64,102.0,95.0,37.17,111.0,76.0,65.0,21.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.92,1,NaN,NaN,-0.03,17,0,9
65,99.0,96.0,37.00,106.0,75.0,64.0,16.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.92,1,NaN,NaN,-0.03,18,0,9
66,99.5,92.0,37.22,122.5,79.5,65.5,16.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.92,1,NaN,NaN,-0.03,19,0,9
67,98.0,93.0,37.00,115.5,73.5,60.0,20.75,NaN,-2.0,23.0,1.00,7.28,52.0,NaN,NaN,13.0,NaN,NaN,108.0,1.2,NaN,130.0,3.2,2.0,4.6,4.5,NaN,NaN,39.1,14.6,31.0,12.1,NaN,137.0,27.92,1,NaN,NaN,-0.03,20,0,9
68,94.5,96.5,37.06,112.0,73.0,61.5,20.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.92,1,NaN,NaN,-0.03,21,0,9
69,91.5,97.0,36.94,113.0,73.0,61.0,18.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.92,1,NaN,NaN,-0.03,22,0,9


In [12]:
train_df.iloc[:50,:]

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,pid
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,1,0,1
1,97.0,95.0,NaN,98.0,75.33,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,2,0,1
2,89.0,99.0,NaN,122.0,86.00,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,3,0,1
3,90.0,95.0,NaN,NaN,NaN,NaN,30.0,NaN,24.0,NaN,NaN,7.36,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,4,0,1
4,103.0,88.5,NaN,122.0,91.33,NaN,24.5,NaN,NaN,NaN,0.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,5,0,1
5,110.0,91.0,NaN,NaN,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,6,0,1
6,108.0,92.0,36.11,123.0,77.00,NaN,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,7,0,1
7,106.0,90.5,NaN,93.0,76.33,NaN,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,8,0,1
8,104.0,95.0,NaN,133.0,88.33,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,9,0,1
9,102.0,91.0,NaN,134.0,87.33,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,10,0,1


In [13]:
test_df.iloc[:50,:]

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,pid
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.82,0,1.0,0.0,-1195.71,1,0,3
1,87.0,96.0,NaN,131.0,70.33,NaN,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.82,0,1.0,0.0,-1195.71,2,0,3
2,93.0,97.0,37.11,130.0,69.33,NaN,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.82,0,1.0,0.0,-1195.71,3,0,3
3,90.0,95.0,NaN,128.0,69.33,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.82,0,1.0,0.0,-1195.71,4,0,3
4,89.0,97.0,NaN,137.0,62.67,50.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.82,0,1.0,0.0,-1195.71,5,0,3
5,84.0,97.0,NaN,144.0,81.00,52.0,19.0,NaN,NaN,32.0,NaN,NaN,NaN,NaN,NaN,31.0,NaN,11.1,99.0,0.9,NaN,51.0,NaN,2.5,2.4,3.8,NaN,NaN,26.9,9.1,NaN,10.0,NaN,488.0,45.82,0,1.0,0.0,-1195.71,6,0,3
6,82.0,97.0,38.33,140.0,77.00,50.0,26.0,NaN,8.0,NaN,0.5,7.51,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.82,0,1.0,0.0,-1195.71,7,0,3
7,84.0,97.0,38.61,135.0,75.00,50.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.82,0,1.0,0.0,-1195.71,8,0,3
8,91.0,95.0,NaN,147.0,85.00,55.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.82,0,1.0,0.0,-1195.71,9,0,3
9,92.0,96.0,NaN,140.0,80.00,52.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.82,0,1.0,0.0,-1195.71,10,0,3


In [14]:
train_df.pid.unique()

array([     1,      2,      4, ..., 119997, 119998, 120000], dtype=int64)

In [15]:
test_df.pid.unique()

array([     3,      9,     16, ..., 119991, 119993, 119999], dtype=int64)

In [16]:
train_df

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,pid
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,1,0,1
1,97.0,95.0,NaN,98.0,75.33,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,2,0,1
2,89.0,99.0,NaN,122.0,86.00,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,3,0,1
3,90.0,95.0,NaN,NaN,NaN,NaN,30.0,NaN,24.0,NaN,NaN,7.36,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,4,0,1
4,103.0,88.5,NaN,122.0,91.33,NaN,24.5,NaN,NaN,NaN,0.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156567,80.0,96.0,NaN,115.0,87.00,65.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.00,0,NaN,NaN,0.00,31,0,120000
1156568,74.0,97.0,NaN,114.0,83.00,67.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.00,0,NaN,NaN,0.00,32,0,120000
1156569,78.0,98.0,NaN,110.0,83.00,69.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.00,0,NaN,NaN,0.00,33,0,120000
1156570,82.0,99.0,36.6,124.0,91.00,71.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.00,0,NaN,NaN,0.00,34,0,120000


### Nan 값 채우기 (median & forward fill)

In [ ]:
vital_cols=['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
       'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
       'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
       'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets']

col_stat = {}

for col in tqdm(vital_cols):
    col_stat[col] = {}
    col_stat[col]['mean']=train_df[col].median()
    col_stat[col]['median'] = train_df[col].median()
    col_stat[col]['std'] = train_df[col].std()
    
    first_index = train_df.drop_duplicates(['pid'], keep='first').index
    train_df.loc[first_index, col] = train_df.loc[first_index,col].apply(lambda x:col_stat[col]['median'] if pd.isna(x) else x)
    train_df[col] = train_df[col].fillna(method='ffill')
    
    first_index = test_df.drop_duplicates(['pid'], keep='first').index
    test_df.loc[first_index, col] = test_df.loc[first_index,col].apply(lambda x:col_stat[col]['median'] if pd.isna(x) else x)
    test_df[col] = test_df[col].fillna(method='ffill')

NEWS score ['HR', 'O2Sat', 'Temp', 'SBP', 'Resp']

In [ ]:
test_df.at[test_df.index[0],'Temp']

In [ ]:
news_score = 0
if test_df.at[test_df.index[0],'Temp'] <= 35.0:
    news_score += 3
elif 35.1 <= test_df.at[test_df.index[0],'Temp'] <= 36.0:
    news_score += 1
elif 36.1 <= test_df.at[test_df.index[0],'Temp'] <= 38.0:
    news_score += 0
elif 38.1 <= test_df.at[test_df.index[0],'Temp'] <= 39.0:
    news_score += 1
elif 39.1 <= test_df.at[test_df.index[0],'Temp']:
    news_score += 1
news_score

In [ ]:
test_df.loc[0]

In [ ]:
# NEWS score 구하기
def get_NEWS(news_input):
    
    news_score = 0
    
    if news_input['Temp'] <= 35.0:
        news_score += 3
    elif 35.1 <= news_input['Temp'] <= 36.0:
        news_score += 1
    elif 36.1 <= news_input['Temp'] <= 38.0:
        news_score += 0
    elif 38.1 <= news_input['Temp'] <= 39.0:
        news_score += 1
    elif 39.1 <= news_input['Temp']:
        news_score += 1
        
    if news_input['SBP'] <= 90:
        news_score += 3
    elif 91 <= news_input['SBP'] <= 100:
        news_score += 2
    elif 101 <= news_input['SBP'] <= 110:
        news_score += 1
    elif 111 <= news_input['SBP'] <= 219:
        news_score += 0
    elif 220 <= news_input['SBP']:
        news_score += 3
        
    if news_input['Resp'] <= 8:
        news_score += 3
    elif 9 <= news_input['Resp'] <= 11:
        news_score += 1
    elif 12 <= news_input['Resp'] <= 20:
        news_score += 0
    elif 21 <= news_input['Resp'] <= 24:
        news_score += 2
    elif 25 <= news_input['Resp']:
        news_score += 3
        
    if news_input['HR'] <= 40:
        news_score += 3
    elif 41 <= news_input['HR'] <= 50:
        news_score += 1
    elif 51 <= news_input['HR'] <= 90:
        news_score += 0
    elif 91 <= news_input['HR'] <= 110:
        news_score += 1
    elif 111 <= news_input['HR'] <= 130:
        news_score += 2
    elif 131 <= news_input['HR']:
        news_score += 3
        
    if news_input['O2Sat'] <= 91:
        news_score += 3
    elif 92 <= news_input['O2Sat'] <= 93:
        news_score += 2
    elif 94 <= news_input['O2Sat'] <= 95:
        news_score += 1
    elif 96 <= news_input['O2Sat']:
        news_score += 0
        
        
    return news_score

test_df['NEWS'] = 0
news_columns = ['Temp','Resp', 'SBP', 'HR', 'O2Sat'] # NEWS에서 쓰는 컬럼 리스트
for i in tqdm(test_df.index):
    news_input = test_df.loc[i, news_columns]
    test_df.at[i, 'NEWS'] = get_NEWS(news_input)

In [ ]:
test_df

In [ ]:
test_df['NEWS'].unique()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# y_pred와 y_true가 주어진다고 가정합니다
fpr, tpr, thresholds = roc_curve(test_df['SepsisLabel'], test_df['NEWS'])
auroc_score = roc_auc_score(test_df['SepsisLabel'], test_df['NEWS'])
print('NEWS')

# AUROC 점수와 ROC 곡선을 출력합니다
print(f"AUROC score: {auroc_score:.4f}")
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()


In [ ]:
test_df['NEWS'].max()

In [ ]:
test_df

In [ ]:
pd.set_option('display.max_rows',None)

In [ ]:
test_df['Creatinine'].value_counts()

In [ ]:
test_df.at[test_df.index[0],'PaCO2']

In [ ]:
x=15

In [ ]:
if 5<x<15:
    print(2)

In [ ]:
5<x<15

In [ ]:
test_df.columns

In [ ]:
def get_pao2(spo2):
    sp_map = {
                85:50,
                86:51,
                87:52,
                88:54,
                89:56,
                90:58,
                91:60,
                92:64,
                93:68,
                94:73,
                95:80,
                96:90,
                97:110
            }
    spo2 = int(round(spo2))
    if spo2 <85:
        spo2=85
    if spo2>97:
        spo2=97
    return sp_map[spo2]

In [ ]:
test_df['PaO2'] = test_df['SaO2'].apply(get_pao2)

In [ ]:
test_df['PaO2']

In [ ]:
def get_SOFA(sofa_input) :
    # ToDo : sofa계산
    sofa_scores = 0
    # Respiratory system score
    if sofa_input['FiO2']>0:
        if 300 <= sofa_input['PaO2']/sofa_input['FiO2'] < 400:
                sofa_scores += 1
        elif 200 <= sofa_input['PaO2']/sofa_input['FiO2'] < 300:
            sofa_scores += 2
        elif 100 <= sofa_input['PaO2']/sofa_input['FiO2'] < 200:
            sofa_scores += 3
        elif sofa_input['PaO2']/sofa_input['FiO2'] < 100:
            sofa_scores += 4

            
    # Coagulation system score
    if 100 <= sofa_input['Platelets'] < 150:
        sofa_scores += 1
    elif 50 <= sofa_input['Platelets'] < 100:
        sofa_scores += 2
    elif 20 <= sofa_input['Platelets'] < 50:
        sofa_scores += 3
    elif sofa_input['Platelets'] < 20:
        sofa_scores += 4
    
    # Liver system score
    if 1.2 <= sofa_input['Bilirubin_total'] < 2.0 :
        sofa_scores += 1
    elif 2.0 <= sofa_input['Bilirubin_total'] < 6.0 :
        sofa_scores += 2
    elif 6.0 <= sofa_input['Bilirubin_total'] < 12.0:
        sofa_scores += 3
    elif 12.0 <= sofa_input['Bilirubin_total']:
        sofa_scores += 4
    
    # Cardiovascular system score
    if sofa_input['MAP'] < 70:
        sofa_scores += 1
        
        
    # Renal system score
    if 1.2 <= sofa_input['Creatinine'] < 2.0:
        sofa_scores += 1
    elif 2.0 <= sofa_input['Creatinine'] < 3.5 :
        sofa_scores += 2
    elif 3.5 <= sofa_input['Creatinine'] < 5.0 :
        sofa_scores += 3
    elif 5.0 <= sofa_input['Creatinine']:
        sofa_scores += 4
        
    return sofa_scores

test_df['SOFA'] = 0
sofa_columns = ['PaO2', 'FiO2','Platelets', 'MAP', 'Bilirubin_total', 'Creatinine'] # SOFA에서 쓰는 컬럼 리스트
sofa_scores = []
for i in tqdm(test_df.index):
    sofa_input = test_df.loc[i, sofa_columns]
    sofa_scores.append(get_SOFA(sofa_input))
test_df['SOFA'] = sofa_scores

In [ ]:
test_df['SOFA'].value_counts()

In [ ]:
fpr, tpr, thresholds = roc_curve(test_df['SepsisLabel'], test_df['SOFA'])
auroc_score = roc_auc_score(test_df['SepsisLabel'], test_df['SOFA'])
print('SOFA')

# AUROC 점수와 ROC 곡선을 출력합니다
print(f"AUROC score: {auroc_score:.4f}")
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

### Normalize

In [ ]:
for col in tqdm(vital_cols) :
    train_df[col] = (train_df[col]-col_stat[col]['mean'])/col_stat[col]['std']

In [ ]:
for col in tqdm(vital_cols):
    mean_value = train_df[col].mean()   

In [ ]:
for col in tqdm(vital_cols):
    std_value = train_df[col].std()   

In [ ]:
for col in tqdm(vital_cols) :
    test_df[col] = (test_df[col]-col_stat[col]['mean'])/col_stat[col]['std']

In [ ]:
train_df.iloc[150:390,:]

In [ ]:
dfA.query('pid==9')[['pid','ICULOS','SepsisLabel']].loc[240]

In [ ]:
train_df.iloc[150:391,:]

In [ ]:
'''
할일 : train_df에서 batch_size만큼 length=30 seq 데이터를 추출해와서 배치 만들어서 모델에 집어넣기
+ 패혈증, 일반 데이터 균등 샘플링
'''

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, WeightedRandomSampler

In [ ]:
def _make_window(rows, group, feature_columns, sequence_length):
        X = []
        for i in range(len(rows)):
            row = rows.iloc[i]

            df = group.get_group((row.pid))
            df = df.loc[:row.name]
            features = df[feature_columns].values

            if len(features) < sequence_length:
                x = np.zeros([sequence_length, len(feature_columns)])
                x[-len(features):] = features
            else:
                x = features[-sequence_length:]

            X.append(x)

        return X
    
class Physionet2019(Dataset):
    def __init__(self, df):
        self.group = df.groupby(['pid'])
        self.seq_len = 30
        self.df = df 
        self.feat_cols = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
                           'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
                           'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
                           'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
                           'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
                           'Fibrinogen', 'Platelets']
        self.info_cols = ['pid', 'HospAdmTime', 'ICULOS', 'SepsisLabel', 'Age','Gender']

    def __len__(self):
        return len(self.df)

    def get_x_y(self, idx, df):
        rows = df.iloc[[idx]]
        x = _make_window(rows, self.group, self.feat_cols, self.seq_len)
        y = list(rows['SepsisLabel'].values)
        return x, y, rows
    
    def __getitem__(self, idx):
        x, y, rows = self.get_x_y(idx, self.df)
        return x, y, list(rows[self.info_cols].values)

In [ ]:
train_ds = Physionet2019(train_df)
test_ds = Physionet2019(test_df)

In [ ]:
train_ds

In [ ]:
x, y, info = test_ds[10] 

In [ ]:
x

In [ ]:
y

In [ ]:
info

In [ ]:
def get_sampler(train_df):
    class_counts = train_df['SepsisLabel'].value_counts() 
    num_samples = sum(class_counts)
    labels = train_df['SepsisLabel'].to_list()

    #클래스별 가중치 부여 [48000/43200, 48000/4800] => class 1에 가중치 높게 부여하게 됨
    class_weights = [num_samples / class_counts[i] for i in range(len(class_counts))] 

    # 해당 데이터의 label에 해당되는 가중치
    weights = [class_weights[labels[i]] for i in range(int(num_samples))] #해당 레이블마다의 가중치 비율
    sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))
    return sampler

In [ ]:
def val_collate(batch):
    X = []
    Label = []
    INFO = []
    for x, label, info in batch:
        for i in range(len(x)):
            X.append(x[i])
            Label.append(label[i])
            INFO.append(info[i])

    X_ = [torch.Tensor(x) for x in X]
    xs = torch.stack(X_, dim=0)
    return xs, torch.Tensor(Label), np.array(INFO)

In [ ]:
train_dl = DataLoader(train_ds, collate_fn=val_collate, sampler=get_sampler(train_df), batch_size=128)

In [ ]:
# 학습
for epoch in range(10):
    for data in train_dl:
        x, y, info = data
        pred = model(x)
        loss = loss_fn(y, pred)
        optimizer.
        
#평가
test_Y, test_pred, test_info = [], [], []
for data in test_dl:
    x, y, info = data
    pred = model(x)
    test_pred.append(pred)
    test_info.append(info)
    
test_info = np.concatenate(test_info)

